<center>
<img src="../../img/ods_stickers.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #6 (demo)
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
**Same assignment as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/a6-demo-linear-models-and-rf-for-regression) + [solution](https://www.kaggle.com/kashnitsky/a6-demo-regression-solution).**    
    
<img src='../../img/wine_quality.jpg' width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [2]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [9]:
data.columns.isin(['quality'])

array([False, False, False, False, False, False, False, False, False,
       False, False,  True])

**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [14]:
X = data.loc[:, ~data.columns.isin(['quality'])]
y = data['quality']

In [16]:
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [18]:
y.head()

0    6
1    6
2    6
3    6
4    6
Name: quality, dtype: int64

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [33]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [34]:
print("Mean squared error (train): %.4f" % mean_squared_error(linreg.predict(X_train_scaled), y_train)) 
print("Mean squared error (test): %.4f" % mean_squared_error(linreg.predict(X_test_scaled), y_test))

Mean squared error (train): 0.5581
Mean squared error (test): 0.5842


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [35]:
linreg_coef = pd.DataFrame({'coef': linreg.coef_, 'feature': X.columns, 'coef_abs': np.abs(linreg.coef_)})
linreg_coef.sort_values(by='coef_abs', ascending=False)

,coef,feature,coef_abs
7,-0.665720,density,0.665720
3,0.538164,residual sugar,0.538164
1,-0.192260,volatile acidity,0.192260
8,0.150036,pH,0.150036
10,0.129533,alcohol,0.129533
0,0.097822,fixed acidity,0.097822
9,0.062053,sulphates,0.062053
5,0.042180,free sulfur dioxide,0.042180
6,0.014304,total sulfur dioxide,0.014304
4,0.008127,chlorides,0.008127


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [36]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

In [38]:
print("Mean squared error (train): %.4f" % mean_squared_error(lasso1.predict(X_train_scaled), y_train)) 
print("Mean squared error (test): %.4f" % mean_squared_error(lasso1.predict(X_test_scaled), y_test))

Mean squared error (train): 0.5638
Mean squared error (test): 0.5737


In [39]:
lasso1.coef_

array([-0.        , -0.18847922, -0.        ,  0.25636281, -0.00274732,
        0.04308762, -0.        , -0.23549185,  0.06727655,  0.02972152,
        0.32242505])

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [42]:
lasso1_coef = pd.DataFrame({'coef': lasso1.coef_, 'feature': X.columns, 'coef_abs': np.abs(lasso1.coef_)})
lasso1_coef.sort_values(by='coef_abs', ascending=False)

,coef,feature,coef_abs
10,0.322425,alcohol,0.322425
3,0.256363,residual sugar,0.256363
7,-0.235492,density,0.235492
1,-0.188479,volatile acidity,0.188479
8,0.067277,pH,0.067277
5,0.043088,free sulfur dioxide,0.043088
9,0.029722,sulphates,0.029722
4,-0.002747,chlorides,0.002747
0,-0.000000,fixed acidity,0.000000
2,-0.000000,citric acid,0.000000


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [51]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5, random_state=17)

alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [52]:
lasso_cv.alpha_

0.0002833096101839324

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [53]:
lassocv_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'feature': X.columns, 'coef_abs': np.abs(lasso_cv.coef_)})
lassocv_coef.sort_values(by='coef_abs', ascending=False)

,coef,feature,coef_abs
7,-0.648161,density,0.648161
3,0.526883,residual sugar,0.526883
1,-0.192049,volatile acidity,0.192049
8,0.146549,pH,0.146549
10,0.137115,alcohol,0.137115
0,0.093295,fixed acidity,0.093295
9,0.060939,sulphates,0.060939
5,0.042698,free sulfur dioxide,0.042698
6,0.012969,total sulfur dioxide,0.012969
4,0.006933,chlorides,0.006933


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [55]:
print("Mean squared error (train): %.4f" % mean_squared_error(lasso_cv.predict(X_train_scaled), y_train)) 
print("Mean squared error (test): %.4f" % mean_squared_error(lasso_cv.predict(X_test_scaled), y_test))

Mean squared error (train): 0.5581
Mean squared error (test): 0.5833


In [56]:
lasso2 = Lasso(alpha=0.0002833096101839324, random_state=17)
lasso2.fit(X_train_scaled, y_train)

print("Mean squared error (train): %.4f" % mean_squared_error(lasso2.predict(X_train_scaled), y_train)) 
print("Mean squared error (test): %.4f" % mean_squared_error(lasso2.predict(X_test_scaled), y_test))

Mean squared error (train): 0.5581
Mean squared error (test): 0.5833


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [57]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [61]:
print("Mean squared error (train): %.4f" % mean_squared_error(forest.predict(X_train_scaled), y_train)) 
print("Mean squared error (test): %.4f" % mean_squared_error(forest.predict(X_test_scaled), y_test))
print("Mean squared error (cv): %.4f" % np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error'))))

Mean squared error (train): 0.0753
Mean squared error (test): 0.4220
Mean squared error (cv): 0.4598


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [65]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(random_state=17), 
                                   param_grid=forest_params, 
                                   scoring='neg_mean_squared_error', 
                                   cv=5, 
                                   n_jobs=-1,
                                   verbose=True)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 320 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 820 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 1520 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 2420 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 3150 out of 3150 | elapsed:   48.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [67]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 19, 'max_features': 7, 'min_samples_leaf': 1},
 -0.4346519750542217)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [69]:
print("Mean squared error (train): %.4f" % mean_squared_error(locally_best_forest.predict(X_train_scaled), y_train)) 
print("Mean squared error (test): %.4f" % mean_squared_error(locally_best_forest.predict(X_test_scaled), y_test))
print("Mean squared error (cv): %.4f" % np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_, X_train_scaled, y_train, scoring='neg_mean_squared_error'))))

Mean squared error (train): 0.0836
Mean squared error (test): 0.4098
Mean squared error (cv): 0.4565


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [70]:
rf_importance = pd.DataFrame({'importance': locally_best_forest.best_estimator_.feature_importances_, 
                            'feature': X.columns})
rf_importance.sort_values(by='importance', ascending=False)

,importance,feature
10,0.224432,alcohol
1,0.119393,volatile acidity
5,0.116147,free sulfur dioxide
8,0.072806,pH
6,0.071318,total sulfur dioxide
3,0.070160,residual sugar
7,0.069367,density
4,0.067982,chlorides
0,0.064268,fixed acidity
2,0.062945,citric acid


**Make conclusions about the perdormance of the explored 3 models in this particular prediction task.**